### Ingest races.csv file

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

### Step 1 - Read the csv file using the spark dataframe reader API

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType

In [0]:
races_schema = StructType(fields = [StructField("raceId", IntegerType(), False),
                                        StructField("year", IntegerType(), True),
                                        StructField("round", IntegerType(), True),
                                        StructField("circuitId", IntegerType(), True),
                                        StructField("name", StringType(), True),
                                        StructField("date", DateType(), True),
                                        StructField("time", StringType(), True),
                                        StructField("url", StringType(), True)                          
])

In [0]:
races_df = spark.read \
.option("header", True) \
.schema(races_schema) \
.csv(f"{raw_folder_path}/races.csv")

### Step 2 - Add ingestion date and race_timestamp to the dataframe

In [0]:
from pyspark.sql.functions import to_timestamp, col, lit, concat

In [0]:
races_with_timestamp_df = add_ingestion_date(races_df) \
                        .withColumn("race_timestamp", to_timestamp(concat(col('date'), lit(' '), col('time')), 'yyyy-MM-dd HH:mm:ss')) \
                        .withColumn("data_source", lit(v_data_source))

### Step - 3 Select only required columns and rename as required.

In [0]:
races_selected_df = races_with_timestamp_df.select(
    col("raceId").alias('race_id'), 
    col("year").alias('race_year'), 
    col("round"),
    col("circuitId").alias('circuit_id'),
    col("name"),
    col("ingestion_date"), 
    col("race_timestamp"),
    col("data_source")
)

### Step 4 - Write the output to processed container in parquet format. 

In [0]:
#races_selected_df.write.mode("overwrite").partitionBy('race_year').parquet(f"{processed_folder_path}/races")

races_selected_df.write.mode("overwrite") \
.partitionBy('race_year') \
.format("parquet") \
.saveAsTable("f1_processed.races")

In [0]:
display(spark.read.parquet("/mnt/formula1pond/processed/races"))

In [0]:
%sql
SELECT * 
FROM f1_processed.races;

In [0]:
dbutils.notebook.exit("Success")